In [ ]:
import pandas as pd
import numpy as np


In [ ]:
proff_df = pd.read_excel('/content/400-professions.xlsx')
vacancy_df = pd.read_csv('/content/v.csv')

In [ ]:
import re
from bs4 import BeautifulSoup as bs
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import nltk

nltk.download('stopwords')

def preprocess(sentence):
    soup = bs(sentence, features="html.parser")
    sentence = soup.get_text()
    soup = bs(sentence, features="html.parser")
    sentence = soup.get_text()

    sentence = str(sentence)
    sentence = sentence.lower()
    sentence = sentence.replace('{html}',"")
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', sentence)
    rem_url = re.sub(r'http\S+', '',cleantext)
    rem_num = re.sub('[0-9]+', '', rem_url)
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(rem_num)
    filtered_words = [w for w in tokens if not w in stopwords.words('russian')]

    return " ".join(filtered_words)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
import pandas as pd
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# !python -m spacy download ru_core_news_sm

In [ ]:
nlp = spacy.load('ru_core_news_sm')

In [ ]:
# def preprocess(text):
#     doc = nlp(text.lower())
#     tokens = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]
#     return ' '.join(tokens)

In [ ]:
vacancy_df['cleaned_vacancy_description'] = vacancy_df['clean_description'].apply(preprocess)

In [ ]:
proff_df['Наименование КК_clean']= proff_df['Наименование КК'].apply(preprocess)

<ipython-input-91-052ffeceb055>:10: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = bs(sentence, features="html.parser")
<ipython-input-91-052ffeceb055>:12: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = bs(sentence, features="html.parser")


In [ ]:
cleaned_professions = proff_df['Наименование КК_clean'].tolist()
cleaned_vacancy_descriptions = vacancy_df['cleaned_vacancy_description'].tolist()

In [ ]:
texts = cleaned_professions + cleaned_vacancy_descriptions

In [179]:
# texts

In [ ]:
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(texts)

In [ ]:
similarity_matrix = cosine_similarity(tfidf_matrix)

In [ ]:
professions_similarity = similarity_matrix[:len(cleaned_professions), len(cleaned_professions):]

In [ ]:
most_similar_professions_indices = professions_similarity.argmax(axis=0)
most_similar_professions = [cleaned_professions[idx] for idx in most_similar_professions_indices]


In [ ]:
vacancy_df['most_similar_profession'] = most_similar_professions

vacancy_df.to_csv('matched_professions_vacancies.csv', index=False)

print("Результаты сохранены в 'matched_professions_vacancies.csv'")

Результаты сохранены в 'matched_professions_vacancies.csv'


In [184]:
vacancy_df[['name', 'most_similar_profession']][350:400]

,name,most_similar_profession
350,Торговый представитель по аптекам и магазинам,менеджер продажам
351,Android Разработчик (Удаленно),разработчик мобильных приложений
352,Уборщик,ветеринарный врач
353,Контролер ОТК,менеджер системе менеджмента качества
354,Контролер службы внутренней безопасности,ветеринарный врач
355,Оператор вахтер,ветеринарный врач
356,Junior Category Manager,ветеринарный врач
357,Помощник бухгалтера по расчету заработной платы,бухгалтер налогам
358,Начальник дорожно-строительного участка,техник дорожного строительства
359,Специалист по снабжению (Снабженец),техник аэродромной службы


In [129]:
vacancy_df['clean_description'][216]

'обязанности диагностика ремонт гарантийный постгарантийный ремонт ит оборудования требования высшее образование специальности опыт работы сервисном центре условия полный рабочий день возможность дополнительного обучения'

In [ ]:
vacancy_df[['name', 'most_similar_profession']][:400]

,name,most_similar_profession
0,Senior Application Support,web разработчик
1,Lead Translator,ветеринарный врач
2,Техничка (уборщик) в детский сад,ветеринарный врач
3,Грузчик-упаковщик/рабочий производственного цеха,менеджер продукции
4,Управляющий кофейней Starbucks (г. Шымкент),инспектор связи
...,...,...
395,Электромонтер,инженер конструктор электрических машин аппаратов
396,Врач-рентгенолог,прочие персонал сфере культуры
397,Мастер-установщик фильтров очистки воды,менеджер работе клиентами
398,Начальник службы безопасности,преподаватель вуза военное дело безопасность


In [128]:
vacancy_df['clean_description'][159]

'описание должности должность предполагает ответственность внедрение техническое сопровождение программного продукта minevision основные необходимые навыки способность разрабатывать качественный обслуживаемый код соответствии спецификацией способность работать команде выраженные аналитические навыки навыки решения задач знание современных технологий хорошие коммуникативные навыки устные письменные включая способность сотрудничать давать разъяснения техническим данным сотрудников отдела продаж руководящего состава которые имеют технического образования знание принципов реляционных баз данных хорошее понимание математики статистики необходимый опыт образование области вычислительных систем образование схожей сфере опыт работы ms sql знание sql опыт использования конструкторов отчетов желателен опыт работы программным обеспечением горногеологической отрасли понимание специфики горногеологической отрасли приветствуется основные обязанности обязанности включают ограничиваются написание скри

In [ ]:
vacancy_df['clean_description'][389]

'обязанности реализация запасных частей камаз ведение складского учета поиск привлечение новых клиентов обработка входящих заявок активный поиск клиентов переговоры заключение договоров требования знание знание устройства м камаз навыки оформления основных коммерческих документов коммерческих предложений договоров писем счетов фактур накладных т д'

# **option with rubert**

In [175]:
# !pip install transformers
# !pip install torch

In [ ]:
import torch
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
tokenizer = BertTokenizer.from_pretrained('DeepPavlov/rubert-base-cased')
model = BertModel.from_pretrained('DeepPavlov/rubert-base-cased')

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
def get_bert_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

In [ ]:
vacancy_df['cleaned_vacancy_description'] = vacancy_df['clean_description'].apply(preprocess)
proff_df['Наименование КК_clean'] = proff_df['Наименование КК'].apply(preprocess)

<ipython-input-91-052ffeceb055>:10: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = bs(sentence, features="html.parser")
<ipython-input-91-052ffeceb055>:12: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = bs(sentence, features="html.parser")


In [115]:
proff_df['embedding'] = proff_df['Наименование КК_clean'].apply(get_bert_embedding)

vacancy_df['embedding'] = vacancy_df['cleaned_vacancy_description'].apply(get_bert_embedding)


In [125]:
def find_best_match(vacancy_embedding, profession_embeddings, profession_names):
    similarity_scores = cosine_similarity([vacancy_embedding], profession_embeddings)[0]
    best_match_index = similarity_scores.argmax()
    return profession_names[best_match_index]

vacancy_df['best_profession_match'] = vacancy_df['embedding'].apply(
    lambda x: find_best_match(x, proff_df['embedding'].tolist(), proff_df['Наименование КК_clean'].tolist())
)


In [178]:
vacancy_df[['name', 'most_similar_profession', 'best_profession_match']][:400]

,name,most_similar_profession,best_profession_match
0,Senior Application Support,web разработчик,ux ui дизайнер
1,Lead Translator,ветеринарный врач,ux ui дизайнер
2,Техничка (уборщик) в детский сад,ветеринарный врач,учитель самопознания начального образования
3,Грузчик-упаковщик/рабочий производственного цеха,менеджер продукции,преподаватель вуза общественные науки
4,Управляющий кофейней Starbucks (г. Шымкент),инспектор связи,преподаватель вуза общественные науки
...,...,...,...
395,Электромонтер,инженер конструктор электрических машин аппаратов,инженер механик системам отопления охлаждения ...
396,Врач-рентгенолог,прочие персонал сфере культуры,врач функциональной диагностики
397,Мастер-установщик фильтров очистки воды,менеджер работе клиентами,преподаватель вуза общественные науки
398,Начальник службы безопасности,преподаватель вуза военное дело безопасность,преподаватель вуза общественные науки


In [170]:
# import pandas as pd
# from transformers import BertTokenizer

# tokenizer = BertTokenizer.from_pretrained('DeepPavlov/rubert-base-cased')

# def count_tokens(text):
#     inputs = tokenizer(text, truncation=True)
#     return len(inputs['input_ids'])

# text = 'техник пусконаладочным работам классифицируется специалист нанят уволен работы приказу генерального директора должность техник вводу эксплуатацию назначается лицо имеющее высшее профессиональное техническое образование опыт работы качестве специалиста вводу эксплуатацию менее лет технический специалист вводу эксплуатацию подотчетен непосредственно инженеру вводу эксплуатацию ведущему инженеру электрику силу специфики своей работы техник вводу эксплуатацию должен знать положения директивы приказы инструкции нормативные материалы рк организации ввода эксплуатацию основные технологические процессы производства предприятии технические условия конструктивные особенности назначение режимы работы оборудования предприятия правила технической эксплуатации методы монтажа наладки настройки электрооборудования контрольные инструменты приборы оборудование используемые осмотре вводе эксплуатацию испытаниях обслуживаемого оборудования опыт работы документацией электрической конфигурации однолинейные чертежи электрические схемы технические руководства испытание ввод эксплуатацию распределительного устройства электрораспределительного оборудования mv lv защитных компонентов пусконаладочные работы испытания подача питания панели шкафы ибп цепи зарядных устройств тестирование подача питания бд освещения малой мощности силовые трансформаторы измерительные приборы также базы данных телекоммуникаций испытания подача электропитания полевых потребителей электроэнергии двигатели клапаны нагреватели т д организация монтажных пусконаладочных ремонтных работ техническое обслуживание оборудования поддержка устранения неполадок электрических проблем связанных электрическим оборудованием кабелями компонентами проверка результатов пусконаладочных испытаний обеспечения безопасной эксплуатации электрической распределительной системы лучшие отечественные зарубежные практики безопасности области ввода эксплуатацию основы трудового законодательства правила нормы охраны труда умение взаимодействовать заказчиком многопрофильными специалистами компании подрядчиками культурная осведомленность способность работать людьми разных национальностей знакомство системой разрешение работу сводки инструментов специфические требования площадкиорганизовать выполнить пусконаладочные работы видов оборудования соответствии методическим нормативным иным инструктивным материалом организации пусконаладочных работ обеспечить своевременный ввод эксплуатацию подготовить измерительные приборы аппаратуру работе выполнить метрологический контроль начать работы устранению дефектов оборудования выявленных вводе эксплуатацию храните техническую документацию монтаже вводе эксплуатацию тестировании оборудования оказание помощи подготовке документации передаче досье наценки т д контролировать сотрудников участвующих вводе эксплуатацию обеспечивать соблюдение требований охраны труда соответствии видом выполняемых работ требования высшее либо средне техническое образование области электрики менее х лет опыта работы пуско наладке оборудовании нефтегазовых месторождениях специфике своей деятельности техник электрик пусконаладке должен знать постановления распоряжения приказы методические нормативные материалы вопросам организации пусконаладочных работ электрооборудования основные технологические процессы технические характеристики конструктивные особенности назначение режимы работы оборудования правила технической эксплуатации методы монтажа регулировки наладки электрического энергетического оборудования контрольные средства приборы устройства применяемые проверке наладке испытаниях обслуживаемого электрического энергетического оборудования порядок методы планирования монтажных наладочных испытательных работ организацию монтажных пусконаладочных ремонтных работ проведения технического обслуживания электрического энергетического оборудования передовой отечественный зарубежный опыт области проведения пусконаладочных работ порядок составления смет проведение работ заявок оборудование материалы запасные части измерительные инструменты приборы порядок разработки оформления технической документации основы организации производства труда управления основы трудового законодательства правила нормы охраны труда организует выполняет работу попусконаладке видов оборудования соответствии методическими нормативными другими руководящими материалами организации пусконаладочных работ обеспечивает своевременный ввод эксплуатацию составляет программы календарные графики проведения пусконаладочных работ испытаний согласовывает заказчиками осуществляет подготовку работе средств измерений аппаратуры выполняет метрологический контроль организует работу персонала обеспечивает рациональное расходование сырья материалов необходимых ввода оборудования систем эксплуатацию обеспечения хода технологического процесса период пусконаладочных работ организует обеспечивает качество ведения работ вносит необходимые коррективы способы методы наладки целью достижения необходимых параметров характеристик работы оборудования систем производит регулировку анализирует данные измерений параметров работы выполняет необходимые расчеты дает заключения пригодности эксплуатации отдельных деталей узлов механизмов систем выявляет причины неисправности инициирует работы устранению дефектов оборудования выявленных выполнении пусконаладочных работ условия вахта проживание питание проезд предоставляет компания'
# num_tokens = count_tokens(text)
# print(f"Количество токенов: {num_tokens}")

In [169]:
# long_descriptions = vacancy_df[vacancy_df['word_count'] > 400][['clean_description', 'word_count']]
# print(f"Количество описаний со словами больше 400: {len(long_descriptions)}")
# print(long_descriptions)


In [150]:
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer

import nltk
nltk.download('punkt')
def summarize_text(text, sentence_count=10):
    parser = PlaintextParser.from_string(text, Tokenizer("russian"))
    summarizer = LsaSummarizer()
    summary = summarizer(parser.document, sentence_count)
    return ' '.join([str(sentence) for sentence in summary])

vacancy_df['summarized_description'] = vacancy_df['cleaned_vacancy_description'].apply(summarize_text)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [174]:
vacancy_df['num_tokens'] = vacancy_df['summarized_description'].apply(count_tokens)

In [168]:
# long_token = vacancy_df[vacancy_df['num_tokens'] > 512][['summarized_description', 'num_tokens']]
# long_token